In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 80*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240430143203
0	Validation losses: 0.4802, 37.1948, 37.6749	Best Loss: 37.6749 (0)	Accuracy: 81.96%
1	Validation losses: 0.4516, 8.8097, 9.2614	Best Loss: 9.2614 (0)	Accuracy: 84.78%
2	Validation losses: 0.4945, 1.0623, 1.5567	Best Loss: 1.5567 (0)	Accuracy: 82.68%
3	Validation losses: 0.5217, 0.4092, 0.9309	Best Loss: 0.9309 (0)	Accuracy: 81.74%
4	Validation losses: 0.5025, 0.3498, 0.8523	Best Loss: 0.8523 (0)	Accuracy: 82.26%
5	Validation losses: 0.5330, 0.3034, 0.8364	Best Loss: 0.8364 (0)	Accuracy: 80.54%
6	Validation losses: 0.5024, 0.2380, 0.7404	Best Loss: 0.7404 (0)	Accuracy: 82.06%
7	Validation losses: 0.6952, 0.2017, 0.8969	Best Loss: 0.7404 (1)	Accuracy: 75.68%
8	Validation losses: 0.5166, 0.2057, 0.7223	Best Loss: 0.7223 (0)	Accuracy: 82.34%
9	Validation losses: 0.5412, 0.1693, 0.7105	Best Loss: 0.7105 (0)	Accuracy: 81.42%
10	Validation losses: 0.5023, 0.1552, 0.6576	Best Loss: 0.6576 (0)	Accuracy: 82.12%
11	Validation losses: 0.48

28	Validation losses: 2.3044, 3.3907, 5.6951	Best Loss: 0.6582 (13)	Accuracy: 9.80%
29	Validation losses: 2.3041, 3.3609, 5.6651	Best Loss: 0.6582 (14)	Accuracy: 10.08%
30	Validation losses: 2.3037, 3.3354, 5.6391	Best Loss: 0.6582 (15)	Accuracy: 10.08%
31	Validation losses: 2.3028, 3.3138, 5.6166	Best Loss: 0.6582 (16)	Accuracy: 11.12%
32	Validation losses: 2.3039, 3.2923, 5.5963	Best Loss: 0.6582 (17)	Accuracy: 9.14%
33	Validation losses: 2.3034, 3.2736, 5.5770	Best Loss: 0.6582 (18)	Accuracy: 10.02%
34	Validation losses: 2.3039, 3.2588, 5.5627	Best Loss: 0.6582 (19)	Accuracy: 9.80%
35	Validation losses: 2.3028, 3.2472, 5.5500	Best Loss: 0.6582 (20)	Accuracy: 9.76%
Early stopping!
Total running time:	 2883
INFO:tensorflow:Restoring parameters from ./models/model_20240430155523.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    17.9499995708%
RANDOM SEED:   1 	LAMBDA_EXP:   -4.2
Now: 202

12	Validation losses: 0.4880, 0.1373, 0.6253	Best Loss: 0.6136 (1)	Accuracy: 84.04%
13	Validation losses: 0.5332, 0.1231, 0.6563	Best Loss: 0.6136 (2)	Accuracy: 81.46%
14	Validation losses: 0.4672, 0.1305, 0.5978	Best Loss: 0.5978 (0)	Accuracy: 83.98%
15	Validation losses: 0.4939, 0.1521, 0.6459	Best Loss: 0.5978 (1)	Accuracy: 82.38%
16	Validation losses: 0.5445, 0.1751, 0.7196	Best Loss: 0.5978 (2)	Accuracy: 82.26%
17	Validation losses: 0.5528, 0.1681, 0.7209	Best Loss: 0.5978 (3)	Accuracy: 81.98%
18	Validation losses: 0.4775, 0.1594, 0.6368	Best Loss: 0.5978 (4)	Accuracy: 83.78%
19	Validation losses: 0.6081, 0.1951, 0.8032	Best Loss: 0.5978 (5)	Accuracy: 80.32%
20	Validation losses: 0.5642, 0.1817, 0.7459	Best Loss: 0.5978 (6)	Accuracy: 81.22%
21	Validation losses: 0.5251, 0.1769, 0.7020	Best Loss: 0.5978 (7)	Accuracy: 82.12%
22	Validation losses: 0.7539, 0.3382, 1.0920	Best Loss: 0.5978 (8)	Accuracy: 74.80%
23	Validation losses: 0.7430, 0.3654, 1.1084	Best Loss: 0.5978 (9)	Accuracy:

6	Validation losses: 0.4607, 0.2400, 0.7006	Best Loss: 0.7006 (0)	Accuracy: 83.90%
7	Validation losses: 0.4721, 0.2187, 0.6908	Best Loss: 0.6908 (0)	Accuracy: 83.66%
8	Validation losses: 0.4092, 0.1986, 0.6078	Best Loss: 0.6078 (0)	Accuracy: 85.30%
9	Validation losses: 0.4412, 0.1715, 0.6127	Best Loss: 0.6078 (1)	Accuracy: 84.44%
10	Validation losses: 0.4846, 0.1479, 0.6325	Best Loss: 0.6078 (2)	Accuracy: 83.02%
11	Validation losses: 0.7065, 0.1420, 0.8485	Best Loss: 0.6078 (3)	Accuracy: 75.86%
12	Validation losses: 0.4504, 0.1149, 0.5653	Best Loss: 0.5653 (0)	Accuracy: 83.54%
13	Validation losses: 0.4415, 0.1246, 0.5662	Best Loss: 0.5653 (1)	Accuracy: 84.96%
14	Validation losses: 0.5095, 0.1094, 0.6189	Best Loss: 0.5653 (2)	Accuracy: 81.82%
15	Validation losses: 0.4444, 0.1059, 0.5503	Best Loss: 0.5503 (0)	Accuracy: 84.62%
16	Validation losses: 0.4460, 0.1031, 0.5491	Best Loss: 0.5491 (0)	Accuracy: 84.14%
17	Validation losses: 0.4205, 0.1013, 0.5218	Best Loss: 0.5218 (0)	Accuracy: 85.

38	Validation losses: 0.3683, 0.0839, 0.4522	Best Loss: 0.4510 (1)	Accuracy: 86.70%
39	Validation losses: 0.3648, 0.0836, 0.4484	Best Loss: 0.4484 (0)	Accuracy: 86.80%
40	Validation losses: 0.3619, 0.0832, 0.4451	Best Loss: 0.4451 (0)	Accuracy: 87.04%
41	Validation losses: 0.3632, 0.0834, 0.4466	Best Loss: 0.4451 (1)	Accuracy: 86.92%
42	Validation losses: 0.3671, 0.0834, 0.4505	Best Loss: 0.4451 (2)	Accuracy: 86.72%
43	Validation losses: 0.3666, 0.0835, 0.4500	Best Loss: 0.4451 (3)	Accuracy: 86.74%
44	Validation losses: 0.3681, 0.0835, 0.4517	Best Loss: 0.4451 (4)	Accuracy: 86.74%
45	Validation losses: 0.3722, 0.0839, 0.4560	Best Loss: 0.4451 (5)	Accuracy: 86.30%
46	Validation losses: 0.3757, 0.0835, 0.4592	Best Loss: 0.4451 (6)	Accuracy: 86.60%
47	Validation losses: 0.3792, 0.0839, 0.4631	Best Loss: 0.4451 (7)	Accuracy: 86.62%
48	Validation losses: 0.3892, 0.0853, 0.4744	Best Loss: 0.4451 (8)	Accuracy: 86.22%
49	Validation losses: 0.3977, 0.0847, 0.4823	Best Loss: 0.4451 (9)	Accuracy:

...

34	Validation losses: 0.4137, 0.1643, 0.5780	Best Loss: 0.5111 (11)	Accuracy: 90.78%
35	Validation losses: 0.5014, 0.1626, 0.6640	Best Loss: 0.5111 (12)	Accuracy: 90.44%
36	Validation losses: 0.5059, 0.1611, 0.6670	Best Loss: 0.5111 (13)	Accuracy: 90.22%
37	Validation losses: 0.5683, 0.1599, 0.7282	Best Loss: 0.5111 (14)	Accuracy: 90.92%
38	Validation losses: 0.6129, 0.1590, 0.7719	Best Loss: 0.5111 (15)	Accuracy: 90.80%
39	Validation losses: 0.6316, 0.1584, 0.7900	Best Loss: 0.5111 (16)	Accuracy: 90.68%
40	Validation losses: 0.6526, 0.1581, 0.8107	Best Loss: 0.5111 (17)	Accuracy: 90.70%
41	Validation losses: 0.6735, 0.1577, 0.8312	Best Loss: 0.5111 (18)	Accuracy: 90.58%
42	Validation losses: 0.6967, 0.1571, 0.8537	Best Loss: 0.5111 (19)	Accuracy: 90.72%
43	Validation losses: 0.6960, 0.1563, 0.8523	Best Loss: 0.5111 (20)	Accuracy: 90.46%
Early stopping!
Total running time:	 3634
INFO:tensorflow:Restoring parameters from ./models/model_20240502091650.ckpt
Batches remaining:     5
Batche

34	Validation losses: 0.4827, 0.1637, 0.6464	Best Loss: 0.5074 (10)	Accuracy: 90.46%
35	Validation losses: 0.5073, 0.1620, 0.6694	Best Loss: 0.5074 (11)	Accuracy: 90.14%
36	Validation losses: 0.5196, 0.1606, 0.6802	Best Loss: 0.5074 (12)	Accuracy: 90.38%
37	Validation losses: 0.5622, 0.1594, 0.7216	Best Loss: 0.5074 (13)	Accuracy: 90.56%
38	Validation losses: 0.6020, 0.1585, 0.7605	Best Loss: 0.5074 (14)	Accuracy: 90.38%
39	Validation losses: 0.6343, 0.1579, 0.7922	Best Loss: 0.5074 (15)	Accuracy: 90.62%
40	Validation losses: 0.6507, 0.1576, 0.8083	Best Loss: 0.5074 (16)	Accuracy: 90.62%
41	Validation losses: 0.6674, 0.1572, 0.8247	Best Loss: 0.5074 (17)	Accuracy: 90.66%
42	Validation losses: 0.7033, 0.1566, 0.8599	Best Loss: 0.5074 (18)	Accuracy: 90.62%
43	Validation losses: 0.7163, 0.1559, 0.8722	Best Loss: 0.5074 (19)	Accuracy: 90.62%
44	Validation losses: 0.7177, 0.1550, 0.8727	Best Loss: 0.5074 (20)	Accuracy: 90.48%
Early stopping!
Total running time:	 3719
INFO:tensorflow:Restori

8	Validation losses: 0.2942, 0.1607, 0.4549	Best Loss: 0.4549 (0)	Accuracy: 89.38%
9	Validation losses: 0.3277, 0.1603, 0.4880	Best Loss: 0.4549 (1)	Accuracy: 88.30%
10	Validation losses: 0.3607, 0.1598, 0.5205	Best Loss: 0.4549 (2)	Accuracy: 86.94%
11	Validation losses: 0.3371, 0.1593, 0.4964	Best Loss: 0.4549 (3)	Accuracy: 86.34%
12	Validation losses: 0.3218, 0.1588, 0.4806	Best Loss: 0.4549 (4)	Accuracy: 88.42%
13	Validation losses: 0.3376, 0.1584, 0.4960	Best Loss: 0.4549 (5)	Accuracy: 87.62%
14	Validation losses: 0.3223, 0.1579, 0.4802	Best Loss: 0.4549 (6)	Accuracy: 88.26%
15	Validation losses: 0.3167, 0.1575, 0.4742	Best Loss: 0.4549 (7)	Accuracy: 88.80%
16	Validation losses: 0.3185, 0.1570, 0.4755	Best Loss: 0.4549 (8)	Accuracy: 88.48%
17	Validation losses: 0.2861, 0.1565, 0.4426	Best Loss: 0.4426 (0)	Accuracy: 90.00%
18	Validation losses: 0.3824, 0.1562, 0.5386	Best Loss: 0.4426 (1)	Accuracy: 87.04%
19	Validation losses: 0.3586, 0.1558, 0.5144	Best Loss: 0.4426 (2)	Accuracy: 8

2	Validation losses: 0.3574, 0.1029, 0.4603	Best Loss: 0.4591 (1)	Accuracy: 87.08%
3	Validation losses: 0.3580, 0.1030, 0.4610	Best Loss: 0.4591 (2)	Accuracy: 86.52%
4	Validation losses: 0.3330, 0.1031, 0.4361	Best Loss: 0.4361 (0)	Accuracy: 87.28%
5	Validation losses: 0.3276, 0.1032, 0.4308	Best Loss: 0.4308 (0)	Accuracy: 87.86%
6	Validation losses: 0.3441, 0.1034, 0.4475	Best Loss: 0.4308 (1)	Accuracy: 87.24%
7	Validation losses: 0.3837, 0.1036, 0.4873	Best Loss: 0.4308 (2)	Accuracy: 85.86%
8	Validation losses: 0.2918, 0.1038, 0.3957	Best Loss: 0.3957 (0)	Accuracy: 89.64%
9	Validation losses: 0.3213, 0.1041, 0.4254	Best Loss: 0.3957 (1)	Accuracy: 88.46%
10	Validation losses: 0.3276, 0.1044, 0.4320	Best Loss: 0.3957 (2)	Accuracy: 88.30%
11	Validation losses: 0.3935, 0.1047, 0.4982	Best Loss: 0.3957 (3)	Accuracy: 83.60%
12	Validation losses: 0.3230, 0.1051, 0.4280	Best Loss: 0.3957 (4)	Accuracy: 88.16%
13	Validation losses: 0.3215, 0.1054, 0.4269	Best Loss: 0.3957 (5)	Accuracy: 87.90%


3	Validation losses: 0.3700, 0.0652, 0.4352	Best Loss: 0.4093 (1)	Accuracy: 86.32%
4	Validation losses: 0.3213, 0.0654, 0.3867	Best Loss: 0.3867 (0)	Accuracy: 87.76%
5	Validation losses: 0.3329, 0.0656, 0.3985	Best Loss: 0.3867 (1)	Accuracy: 87.72%
6	Validation losses: 0.3369, 0.0659, 0.4028	Best Loss: 0.3867 (2)	Accuracy: 87.52%
7	Validation losses: 0.3300, 0.0662, 0.3962	Best Loss: 0.3867 (3)	Accuracy: 88.22%
8	Validation losses: 0.2968, 0.0665, 0.3633	Best Loss: 0.3633 (0)	Accuracy: 89.80%
9	Validation losses: 0.3334, 0.0669, 0.4003	Best Loss: 0.3633 (1)	Accuracy: 87.88%
10	Validation losses: 0.3312, 0.0673, 0.3985	Best Loss: 0.3633 (2)	Accuracy: 87.72%
11	Validation losses: 0.3862, 0.0678, 0.4540	Best Loss: 0.3633 (3)	Accuracy: 85.20%
12	Validation losses: 0.3381, 0.0683, 0.4064	Best Loss: 0.3633 (4)	Accuracy: 87.88%
13	Validation losses: 0.3217, 0.0689, 0.3906	Best Loss: 0.3633 (5)	Accuracy: 88.00%
14	Validation losses: 0.3185, 0.0695, 0.3880	Best Loss: 0.3633 (6)	Accuracy: 88.42%